In [ ]:
import pandas as pd

In [ ]:
# set plotly as the default plotting library
pd.options.plotting.backend = "plotly"

In [ ]:
dataf = (
    pd.read_csv("db/transactions23.csv", sep=";", encoding="windows-1252")
    # cast 'Dato to datetime
    .assign(Dato=lambda d: pd.to_datetime(d["Dato"], format="%d.%m.%Y"))
    .reset_index(names="trans_order")
    .drop(columns=["Status", "Afstemt"])
    # remove '.' from 'Saldo' and switch ',' to '.'
    .assign(Saldo=lambda d: d["Saldo"].str.replace(".", "").str.replace(",", ".").astype(float))
    .assign(Beløb=lambda d: d["Beløb"].str.replace(".", "").str.replace(",", ".").astype(float))
    .assign(MontYear=lambda d: d["Dato"].dt.strftime("%m-%Y"))
    
)


In [ ]:
(
    dataf
     .groupby("Dato")
    .agg(
        {
            # "Dato": "first",
            "Saldo": "last",
        }
    )
    .reset_index()


    .plot(
        x="Dato", y="Saldo", kind="line", title="Saldo over tid", labels={"Saldo": "Saldo (DKK)"}
    )
)

In [ ]:
dataf


In [ ]:
cat_expenses = dataf.groupby(["Kategori", "Underkategori"]).agg(
    {
        "Beløb": "sum",
    }
).reset_index()[lambda d: d["Beløb"] < 0].assign(Beløb=lambda d: d["Beløb"].abs())

In [ ]:

import plotly.express as px
fig = px.treemap(
    (
        dataf
        .groupby(["Kategori", "Underkategori"]).agg({"Beløb": "sum"})
        .reset_index()
        [lambda d: d["Beløb"] < 0]
        .assign(Beløb=lambda d: d["Beløb"].abs())
    ),
    path=[px.Constant("all expenses"), "Kategori", "Underkategori"],
    values= "Beløb",
    width=2000, height=600,
    # text size

)
fig.data[0].textinfo = 'label+percent parent+percent root'
fig.update_traces(root_color="white")
fig.update_traces(marker=dict(cornerradius=5))
fig.update_layout(
    margin = dict(t=50, l=25, r=25, b=25),
    font=dict(
        size=20,
    ),

    # textinfo size, show only if > 1%
    uniformtext=dict(minsize=10, mode=False),
)
fig.show()
